In [1]:
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss

import sys
sys.path.append('../kaggle_prediction_library/')

from validation import run_evaluation_framework


In [2]:
to_predict_mens = pd.read_csv('to_predict_mens.csv')
teams = pd.read_csv('../Data/MTeams.csv')
teams1 = teams.rename(columns = {"TeamID": "Team1", "TeamName":"Team1_Name"})
to_predict_mens = to_predict_mens.merge(teams1, how="left", on="Team1")
teams2 = teams.rename(columns = {"TeamID": "Team2", "TeamName":"Team2_Name"})
to_predict_mens = to_predict_mens.merge(teams2, how="left", on="Team2")
kaggle_df = to_predict_mens[to_predict_mens.type == "Historical"]

In [3]:
first_round_odds_data = pd.read_csv('../data/first_round_odds_data.csv')
first_round_odds_data["Date"] = pd.to_datetime(first_round_odds_data["Date"], format='%b %d, %Y')
first_round_odds_data['year'] = first_round_odds_data["Date"].dt.year

kaggle_df["year"] = kaggle_df["ID"].apply(lambda x: x[0:4])
kaggle_df["year"] = kaggle_df["year"].astype(int)

first_round_odds_data1 = first_round_odds_data.rename(columns={"kaggle_team": "Team1_Name",
                                                               "odds": "odds_team1"})
kaggle_df = kaggle_df.merge(first_round_odds_data1, how="left", on=["year", "Team1_Name"])

first_round_odds_data2 = first_round_odds_data.rename(columns={"kaggle_team": "Team2_Name",
                                                               "odds": "odds_team2"
                                                               })
kaggle_df = kaggle_df.merge(first_round_odds_data2, how="left", on=["year", "Team2_Name"])

kaggle_df["final_odds"] = np.where(kaggle_df.odds_team1.isna(),
                                    kaggle_df.odds_team2 * -1, 
                                    kaggle_df.odds_team1 )

final_df = kaggle_df[(~kaggle_df.final_odds.isna())
                     & (~kaggle_df.Total_x.isna())
                     ].copy()

/var/folders/2g/465yxy_x4g786jx2llr5xqh40000gn/T/ipykernel_6650/2537159113.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_df["year"] = kaggle_df["ID"].apply(lambda x: x[0:4])
/var/folders/2g/465yxy_x4g786jx2llr5xqh40000gn/T/ipykernel_6650/2537159113.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle_df["year"] = kaggle_df["year"].astype(int)


In [4]:
def split_first_round(df):

   first_round_df = df[

      ( (df.t1_Seed == 1) & (df.t2_Seed == 16) ) |
      ( (df.t1_Seed == 2) & (df.t2_Seed == 15) ) |
      ( (df.t1_Seed == 3) & (df.t2_Seed == 14) ) |
      ( (df.t1_Seed == 4) & (df.t2_Seed == 13) ) |
      ( (df.t1_Seed == 5) & (df.t2_Seed == 12) ) |
      ( (df.t1_Seed == 6) & (df.t2_Seed == 11) ) |
      ( (df.t1_Seed == 7) & (df.t2_Seed == 10) ) |
      ( (df.t1_Seed == 8) & (df.t2_Seed == 9) ) |
      ( (df.t1_Seed == 9) & (df.t2_Seed == 8) ) |
      ( (df.t1_Seed == 10) & (df.t2_Seed == 7) ) |
      ( (df.t1_Seed == 11) & (df.t2_Seed == 6) ) |
      ( (df.t1_Seed == 12) & (df.t2_Seed == 5) ) |
      ( (df.t1_Seed == 13) & (df.t2_Seed == 4) ) |
      ( (df.t1_Seed == 14) & (df.t2_Seed == 3) ) |
      ( (df.t1_Seed == 15) & (df.t2_Seed == 2) ) |
      ( (df.t1_Seed == 16) & (df.t2_Seed == 1) ) 
   ].copy()

   other_rounds_df = df[

        ~(
      ( (df.t1_Seed == 1) & (df.t2_Seed == 16) ) |
      ( (df.t1_Seed == 2) & (df.t2_Seed == 15) ) |
      ( (df.t1_Seed == 3) & (df.t2_Seed == 14) ) |
      ( (df.t1_Seed == 4) & (df.t2_Seed == 13) ) |
      ( (df.t1_Seed == 5) & (df.t2_Seed == 12) ) |
      ( (df.t1_Seed == 6) & (df.t2_Seed == 11) ) |
      ( (df.t1_Seed == 7) & (df.t2_Seed == 10) ) |
      ( (df.t1_Seed == 8) & (df.t2_Seed == 9) ) |
      ( (df.t1_Seed == 9) & (df.t2_Seed == 8) ) |
      ( (df.t1_Seed == 10) & (df.t2_Seed == 7) ) |
      ( (df.t1_Seed == 11) & (df.t2_Seed == 6) ) |
      ( (df.t1_Seed == 12) & (df.t2_Seed == 5) ) |
      ( (df.t1_Seed == 13) & (df.t2_Seed == 4) ) |
      ( (df.t1_Seed == 14) & (df.t2_Seed == 3) ) |
      ( (df.t1_Seed == 15) & (df.t2_Seed == 2) ) |
      ( (df.t1_Seed == 16) & (df.t2_Seed == 1) )
        ) 
   ].copy()

   return first_round_df, other_rounds_df

In [76]:
first_round_df, other_rounds_df = split_first_round(final_df)

In [77]:
# Define the classifier and parameter grid
model = LogisticRegression()
param_grid = {
    'logisticregression__C': [.005, 0.001, .05, 0.01, 0.1],
}
pipeline = make_pipeline(StandardScaler(), model)




In [7]:
features = [
            #'seed_diff', 
              # 't1_adj_margin', 't2_adj_margin',
              #   't1_final_rank', 't2_final_rank',
              #   't1_OrdinalRank', 't2_OrdinalRank',
                'final_odds', 
                #'Total_x'
                ]

target="Outcome"

In [87]:
def feature_set_evaluator(feature_sets, df, model, param_grid):
   
    eval_dfs = []

    for i, f in tqdm.tqdm(enumerate(feature_sets)):
        eval_df = run_evaluation_framework(df, model, f, param_grid)
        eval_df["feature_set"] = [f]
        eval_dfs.append(eval_df)

    all_scores = pd.concat(eval_dfs, axis=0)
    
    return all_scores


In [92]:
f0 = ['seed_diff', 
    't1_adj_margin', 't2_adj_margin',
    't1_final_rank', 't2_final_rank',
    't1_OrdinalRank', 't2_OrdinalRank']

f1 = ['seed_diff', 
    't1_adj_margin', 't2_adj_margin',
    't1_final_rank', 't2_final_rank',
    't1_OrdinalRank', 't2_OrdinalRank',
    'final_odds', 
    'Total_x']

f2 = ['seed_diff', 
    't1_adj_margin', 't2_adj_margin',
    't1_final_rank', 't2_final_rank',
    't1_OrdinalRank', 't2_OrdinalRank',
    'final_odds']

f3 = ['t1_adj_margin', 't2_adj_margin',
    't1_final_rank', 't2_final_rank',
    't1_OrdinalRank', 't2_OrdinalRank',
    'final_odds']

f4 = ['t1_adj_margin', 't2_adj_margin',
    't1_final_rank', 't2_final_rank',
    't1_OrdinalRank', 't2_OrdinalRank']

f5 = ['t1_final_rank', 't2_final_rank',
    't1_OrdinalRank', 't2_OrdinalRank']

f6 = ['t1_final_rank', 't2_final_rank']

f7 = ['t1_OrdinalRank', 't2_OrdinalRank']

f8 = ['t1_adj_margin', 't2_adj_margin']

f9 = ['t1_final_rank', 't2_final_rank',
      't1_adj_margin', 't2_adj_margin']

f10 = ['final_odds']

f11 = ['t1_OrdinalRank', 't2_OrdinalRank',
      't1_adj_margin', 't2_adj_margin']

f12 = ['t1_OrdinalRank', 't2_OrdinalRank',
      't1_adj_margin', 't2_adj_margin', 'final_odds']

eval_dfs = []

import tqdm

all_scores_df = feature_set_evaluator([f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12], final_df, pipeline, param_grid)



0it [00:00, ?it/s]../kaggle_prediction_library/validation.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["y_prob"] = y_prob[:,1]
../kaggle_prediction_library/validation.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["y_true"] = y_test
../kaggle_prediction_library/validation.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [93]:
pd.set_option('display.max_colwidth', None)

all_scores_df["rolling_season_cv_rank"] = all_scores_df["rolling_season_cv"].rank()
all_scores_df["mean_repeated_cv_score_rank"] = all_scores_df["mean_repeated_cv_score"].rank(ascending=False)
all_scores_df.sort_values(by="mean_repeated_cv_score", ascending=False)


,best_params,mean_repeated_cv_score,repeated_cv_confidence_interval,rolling_season_cv,feature_set,rolling_season_cv_rank,mean_repeated_cv_score_rank
0,{'logisticregression__C': 0.05},-0.182841,"(-0.18426478327072537, -0.1814177861776386)",0.183401,"[t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank]",3.0,1.0
0,{'logisticregression__C': 0.05},-0.183041,"(-0.18453126516123108, -0.18155110736543062)",0.183389,"[t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank, final_odds]",2.0,2.0
0,{'logisticregression__C': 0.05},-0.183052,"(-0.18437988146085116, -0.1817240272300787)",0.183981,"[seed_diff, t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank]",5.0,3.0
0,{'logisticregression__C': 0.05},-0.183137,"(-0.18462516859043918, -0.1816485822642692)",0.183956,"[seed_diff, t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank, final_odds]",4.0,4.0
0,{'logisticregression__C': 0.05},-0.183333,"(-0.18475604829326786, -0.18190976200597403)",0.184049,"[seed_diff, t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank, final_odds, Total_x]",6.0,5.0
0,{'logisticregression__C': 0.1},-0.183390,"(-0.1848274016320886, -0.18195316368810693)",0.183334,"[t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank]",1.0,6.0
0,{'logisticregression__C': 0.05},-0.183682,"(-0.18504644048838334, -0.18231786423668836)",0.184766,"[t1_final_rank, t2_final_rank, t1_adj_margin, t2_adj_margin]",7.0,7.0
0,{'logisticregression__C': 0.1},-0.184372,"(-0.18569534225974807, -0.18304960280001323)",0.184855,"[t1_final_rank, t2_final_rank]",8.0,8.0
0,{'logisticregression__C': 0.05},-0.184673,"(-0.18604685563921794, -0.18329859760340764)",0.184984,"[t1_OrdinalRank, t2_OrdinalRank, t1_adj_margin, t2_adj_margin, final_odds]",9.0,9.0
0,{'logisticregression__C': 0.05},-0.184775,"(-0.18616253160318252, -0.18338701343882763)",0.185235,"[t1_OrdinalRank, t2_OrdinalRank, t1_adj_margin, t2_adj_margin]",10.0,10.0


In [90]:
pd.set_option('display.max_colwidth', None)

all_scores_df["rolling_season_cv_rank"] = all_scores_df["rolling_season_cv"].rank()
all_scores_df["mean_repeated_cv_score_rank"] = all_scores_df["mean_repeated_cv_score"].rank(ascending=False)
all_scores_df.sort_values(by="mean_repeated_cv_score", ascending=False)


,best_params,mean_repeated_cv_score,repeated_cv_confidence_interval,rolling_season_cv,feature_set,rolling_season_cv_rank,mean_repeated_cv_score_rank
0,{'logisticregression__C': 0.05},-0.197002,"(-0.19891571931054028, -0.1950881367588139)",0.194298,"[t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank]",2.0,1.0
0,{'logisticregression__C': 0.05},-0.197395,"(-0.19928851890203841, -0.19550204100860635)",0.194425,"[t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank, final_odds]",3.0,2.0
0,{'logisticregression__C': 0.05},-0.197457,"(-0.19936880385681816, -0.19554461234004128)",0.194655,"[seed_diff, t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank]",4.0,3.0
0,{'logisticregression__C': 0.1},-0.197468,"(-0.19950194817894218, -0.19543483839086212)",0.193560,"[t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank]",1.0,4.0
0,{'logisticregression__C': 0.05},-0.197674,"(-0.19968027227841295, -0.1956675237040288)",0.194739,"[seed_diff, t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank, final_odds]",5.0,5.0
0,{'logisticregression__C': 0.05},-0.197883,"(-0.1999355014097278, -0.1958299552249481)",0.195051,"[seed_diff, t1_adj_margin, t2_adj_margin, t1_final_rank, t2_final_rank, t1_OrdinalRank, t2_OrdinalRank, final_odds, Total_x]",6.0,6.0
0,{'logisticregression__C': 0.05},-0.198133,"(-0.19990888231452209, -0.1963578149527963)",0.196447,"[t1_final_rank, t2_final_rank, t1_adj_margin, t2_adj_margin]",8.0,7.0
0,{'logisticregression__C': 0.1},-0.198638,"(-0.20050360919229104, -0.1967720499481243)",0.195946,"[t1_final_rank, t2_final_rank]",7.0,8.0
0,{'logisticregression__C': 0.1},-0.199504,"(-0.20148948267236275, -0.1975187810391054)",0.197162,"[t1_OrdinalRank, t2_OrdinalRank, t1_adj_margin, t2_adj_margin]",9.0,9.0
0,{'logisticregression__C': 0.1},-0.199811,"(-0.20177845643399453, -0.19784273951754985)",0.197299,"[t1_OrdinalRank, t2_OrdinalRank, t1_adj_margin, t2_adj_margin, final_odds]",10.0,10.0


In [91]:
(-0.197002 + -0.166889) / 2	

-0.1819455

In [94]:
(0.194298 + 0.169773) / 2

0.18203550000000002

In [99]:
percent_first_round = first_round_df.shape[0] / final_df.shape[0]

print(percent_first_round * (-0.166889) + (1 - percent_first_round) * (-0.197002))
print(percent_first_round * (0.169773) + (1 - percent_first_round) * (0.194298))


-0.18212645254869458
0.18218287360132618


In [103]:
print(f"Improvement to repeated cv score: {0.18212645254869458 - .182841}")
print(f"Improvement to rolling season score: {0.18218287360132618 - 0.183401}")


Improvement to repeated cv score: -0.0007145474513054195
Improvement to rolling season score: -0.00121812639867383


In [114]:
kaggle_df[kaggle_df.t1_Seed == 1]

,Unnamed: 0,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,...,Home Spread_y,ATS_y,Total_y,OU_y,Round_y,Away Seed_y,Home Seed_y,home_flag_y,odds_team2,final_odds
1,1,Historical,2003_1112_1436,NaN,2003,1112,1436,1.0,M,29.0,...,-25.5,L,142.5,U,1.0,16.0,1.0,0.0,25.5,-25.5
13,13,Historical,2003_1328_1354,NaN,2003,1328,1354,1.0,M,17.0,...,-27.5,W,133.5,U,1.0,16.0,1.0,0.0,27.5,-27.5
21,21,Historical,2003_1246_1237,NaN,2003,1246,1237,1.0,M,31.0,...,-25.5,L,137.5,O,1.0,16.0,1.0,0.0,25.5,-25.5
29,29,Historical,2003_1400_1421,NaN,2003,1400,1421,1.0,M,21.0,...,-28.5,W,154.0,U,1.0,16.0,1.0,0.0,28.5,-28.5
33,33,Historical,2003_1112_1211,NaN,2003,1112,1211,1.0,M,1.0,...,-1.5,W,134.0,O,1.0,9.0,8.0,0.0,1.5,-25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,2425,Historical,2021_1276_1417,NaN,2021,1276,1417,0.0,M,-2.0,...,-3.5,W,139.0,U,1.0,11.0,6.0,0.0,3.5,-24.5
2428,2428,Historical,2021_1211_1124,NaN,2021,1211,1124,0.0,M,-16.0,...,-25.5,L,138.5,U,1.0,16.0,1.0,1.0,-25.5,-33.0
2469,2469,Historical,2022_1124_1314,NaN,2022,1124,1314,0.0,M,-7.0,...,-4.0,W,153.5,O,1.0,9.0,8.0,1.0,-4.0,-20.5
2481,2481,Historical,2022_1211_1116,NaN,2022,1211,1116,0.0,M,-6.0,...,-5.0,L,140.0,O,1.0,13.0,4.0,1.0,-5.0,-22.5


In [115]:
kaggle_df[(kaggle_df["Team1"] == 1112)
          & (kaggle_df["Season"] == 2003)][['t1_Seed',
 't2_Seed', 't1_final_rank', 't2_final_rank']]


,t1_Seed,t2_Seed,t1_final_rank,t2_final_rank
1,1.0,16.0,95.514642,72.542565
33,1.0,9.0,95.514642,83.471755
49,1.0,5.0,95.514642,87.665786
1305,1.0,2.0,95.514642,93.380430


In [123]:
1 - (6/16), 1 - (14/16),

(0.625, 0.125)

In [109]:
list(kaggle_df.columns)

['Unnamed: 0',
 'type',
 'ID',
 'Pred',
 'Season',
 'Team1',
 'Team2',
 'Outcome',
 'Gender',
 'margin',
 't1_quality',
 't2_quality',
 't1_FGM',
 't1_FGA',
 't1_FGM3',
 't1_FGA3',
 't1_OR',
 't1_Ast',
 't1_TO',
 't1_Stl',
 't1_PF',
 't1_PointDiff',
 't2_FGM',
 't2_FGA',
 't2_FGM3',
 't2_FGA3',
 't2_OR',
 't2_Ast',
 't2_TO',
 't2_Stl',
 't2_PF',
 't2_PointDiff',
 't1_team_rating_538',
 't1_rd1_win',
 't1_rd2_win',
 't1_rd3_win',
 't1_rd4_win',
 't1_rd5_win',
 't1_rd6_win',
 't1_rd7_win',
 't2_team_rating_538',
 't2_rd1_win',
 't2_rd2_win',
 't2_rd3_win',
 't2_rd4_win',
 't2_rd5_win',
 't2_rd6_win',
 't2_rd7_win',
 't1_OrdinalRank',
 't2_OrdinalRank',
 't1_Seed',
 't2_Seed',
 'seed_diff',
 't1_adj_oe',
 't1_adj_de',
 't1_adj_margin',
 't2_adj_oe',
 't2_adj_de',
 't2_adj_margin',
 't1_final_rank',
 't2_final_rank',
 't1_adjem',
 't1_adjo',
 't1_adjo_rank',
 't1_adjd',
 't1_adjd_rank',
 't1_adjt',
 't1_adjt_rank',
 't1_luck',
 't1_luck_rank',
 't1_sos_adjem',
 't1_sos_adjem_rank',
 't1_so

In [11]:
# Define the classifier and parameter grid
from validation import rolling_season_cv

features = f0
model = LogisticRegression(C=0.01)
pipeline = make_pipeline(StandardScaler(), model)
avg_validation_score_f0, all_preds_f0 = rolling_season_cv(model, first_round_df, features, label='Outcome', cv_start=2007, return_preds=True)


../kaggle_prediction_library/validation.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["y_prob"] = y_prob[:,1]
../kaggle_prediction_library/validation.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["y_true"] = y_test
../kaggle_prediction_library/validation.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [12]:
all_preds_f0

,Unnamed: 0,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,...,Total_y,OU_y,Round_y,Away Seed_y,Home Seed_y,home_flag_y,odds_team2,final_odds,y_prob,y_true
257,257,Historical,2007_1130_1403,NaN,2007,1130,1403,1.0,M,9.0,...,135.5,O,1.0,10.0,7.0,0.0,3.0,-3.0,0.622429,1.0
258,258,Historical,2007_1139_1330,NaN,2007,1139,1330,1.0,M,11.0,...,123.5,U,1.0,12.0,5.0,0.0,2.0,-2.0,0.809220,1.0
259,259,Historical,2007_1207_1125,NaN,2007,1207,1125,1.0,M,25.0,...,126.0,O,1.0,15.0,2.0,0.0,16.5,-16.5,0.959212,1.0
260,260,Historical,2007_1231_1211,NaN,2007,1231,1211,1.0,M,13.0,...,141.0,U,1.0,10.0,7.0,0.0,1.5,-1.5,0.694022,1.0
261,261,Historical,2007_1257_1390,NaN,2007,1257,1390,1.0,M,20.0,...,130.5,O,1.0,11.0,6.0,0.0,6.0,-6.0,0.739513,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2459,Historical,2022_1463_1345,NaN,2022,1463,1345,0.0,M,-22.0,...,144.5,U,1.0,14.0,3.0,1.0,-16.5,16.5,0.083724,0.0
2460,2460,Historical,2022_1371_1395,NaN,2022,1371,1395,0.0,M,-27.0,...,128.5,U,1.0,9.0,8.0,0.0,1.5,-1.5,0.548213,0.0
2461,2461,Historical,2022_1439_1400,NaN,2022,1439,1400,0.0,M,-8.0,...,125.0,O,1.0,11.0,6.0,1.0,-1.5,1.5,0.369526,0.0
2463,2463,Historical,2022_1174_1437,NaN,2022,1174,1437,0.0,M,-20.0,...,135.5,O,1.0,15.0,2.0,1.0,-15.0,15.0,0.064376,0.0


In [13]:
# Define the classifier and parameter grid
from validation import rolling_season_cv

features = f10
model = LogisticRegression(C=0.1)
pipeline = make_pipeline(StandardScaler(), model)
avg_validation_score_f10, all_preds_f10 = rolling_season_cv(model, first_round_df, features, label='Outcome', cv_start=2007, return_preds=True)



../kaggle_prediction_library/validation.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["y_prob"] = y_prob[:,1]
../kaggle_prediction_library/validation.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["y_true"] = y_test
../kaggle_prediction_library/validation.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [27]:
all_preds_f10.iloc[1,:][["Team1_Name", "Team2_Name", "Season", "y_prob"]]

Team1_Name          Butler
Team2_Name    Old Dominion
Season                2007
y_prob            0.595693
Name: 258, dtype: object

In [28]:
all_preds_f0["y_prob_f10"] = all_preds_f10["y_prob"].copy()

In [30]:
all_preds_f0["diff"] = abs(all_preds_f0["y_prob_f10"] - all_preds_f0["y_prob"])

In [40]:
all_preds_f0.sort_values(by="diff", ascending=False)[["Team1_Name", "Team2_Name", "Season", 
                                                      "final_odds", "y_prob", "y_prob_f10", "diff"]]

,Team1_Name,Team2_Name,Season,final_odds,y_prob,y_prob_f10,diff
1658,Tennessee,Oklahoma St,2009,-3.0,0.394096,0.641290,0.247195
410,Oklahoma St,Tennessee,2009,3.0,0.607579,0.364436,0.243142
1506,Old Dominion,Butler,2007,2.0,0.192947,0.415319,0.222372
945,Rhode Island,Creighton,2017,-2.0,0.358912,0.580531,0.221619
2193,Creighton,Rhode Island,2017,2.0,0.640192,0.421474,0.218718
...,...,...,...,...,...,...,...
671,Indiana,James Madison,2013,-21.5,0.973195,0.973172,0.000023
1633,Chattanooga,Connecticut,2009,20.5,0.020357,0.020376,0.000020
747,Virginia,Coastal Car,2014,-20.0,0.961393,0.961402,0.000009
996,Rhode Island,Oklahoma,2018,-1.5,0.562869,0.562861,0.000008


In [74]:
all_preds_f0["average_preds"] = (all_preds_f0["y_prob"] * .03 + all_preds_f0["y_prob_f10"] * .96)

In [75]:
from sklearn.metrics import brier_score_loss

brier_score_loss(all_preds_f0["Outcome"], all_preds_f0["average_preds"])

#0.1702145057142229
#0.17021374444026996
#0.17021373159588687
#0.17021446718107358

0.1700891731604283

In [43]:
brier_score_loss(all_preds_f0["Outcome"], all_preds_f0["y_prob_f10"])

0.17021827355083813

In [44]:
brier_score_loss(all_preds_f0["Outcome"], all_preds_f0["y_prob"])

0.17369718661212352

In [32]:
all_scores_df["rolling_season_cv_rank"] = all_scores_df["rolling_season_cv"].rank()
all_scores_df["mean_repeated_cv_score_rank"] = all_scores_df["mean_repeated_cv_score"].rank(ascending=False)

all_scores_df.sort_values(by="mean_repeated_cv_score", ascending=False)

,best_params,mean_repeated_cv_score,repeated_cv_confidence_interval,rolling_season_cv,feature_set,rolling_season_cv_rank,mean_repeated_cv_score_rank
0,{'logisticregression__C': 0.1},-0.166889,"(-0.16895556189837432, -0.16482223487707984)",0.169773,f10,2.0,1.0
0,{'logisticregression__C': 0.1},-0.167167,"(-0.16903528127920922, -0.16529812772842703)",0.169774,f9,3.0,2.0
0,{'logisticregression__C': 0.1},-0.167197,"(-0.16931148889068726, -0.1650820133042285)",0.170618,f8,6.0,3.0
0,{'logisticregression__C': 0.1},-0.167231,"(-0.16922183521443218, -0.16524099382994373)",0.171004,f6,10.0,4.0
0,{'logisticregression__C': 0.01},-0.167444,"(-0.16940203034668902, -0.16548572268137682)",0.170731,f2,7.0,5.0
0,{'logisticregression__C': 0.05},-0.167691,"(-0.16984986105368596, -0.1655313209175104)",0.170447,f3,5.0,6.0
0,{'logisticregression__C': 0.1},-0.167748,"(-0.16988793517157408, -0.16560803417408576)",0.169412,f7,1.0,7.0
0,{'logisticregression__C': 0.01},-0.167749,"(-0.16976349409923197, -0.1657344917981467)",0.170731,f1,8.0,8.0
0,{'logisticregression__C': 0.05},-0.167832,"(-0.16985402717949807, -0.16581043329123255)",0.170351,f5,4.0,9.0
0,{'logisticregression__C': 0.01},-0.167988,"(-0.16989393511203152, -0.1660819858309456)",0.171363,f0,11.0,10.0


In [28]:
all_scores_df["rolling_season_cv"].rank()

0    11.0
0     8.0
0     7.0
0     5.0
0     9.0
0     4.0
0    10.0
0     1.0
0     6.0
0     3.0
0     2.0
0    12.0
Name: rolling_season_cv, dtype: float64

In [24]:
first_round_df.shape[0] / final_df.shape[0], other_rounds_df.shape[0] / final_df.shape[0]

(0.4939908827186075, 0.5060091172813924)

In [96]:
eval_df

,best_params,mean_repeated_cv_score,repeated_cv_confidence_interval,rolling_season_cv
0,{'C': 0.001},-0.184513,"(-0.18582705050235124, -0.18319855591261971)",0.185355


In [93]:
eval_df



,best_params_,mean_repeated_cv_score,repeated_cv_confidence_interval,rolling_season_cv
0,{'C': 0.001},-0.184473,"(-0.1859400105195753, -0.1830052942078014)",0.185355


In [ ]:
# Define the classifier and parameter grid
classifier = LogisticRegression()
param_grid = {
    'C': [.005, 0.001, .05, 0.01, 0.1],
}


In [72]:
rows = []
for i in range(250):
    row = grid_search.cv_results_[f'split{i}_test_score'][grid_search.best_index_]
    rows.append(row)

tmp = pd.DataFrame(rows, columns = ["best_scores"])

mean_score, confidence_interval = calculate_confidence_interval(tmp, "best_scores")


In [74]:
mean_score, confidence_interval

(-0.18441025872288805, (-0.18581356308446836, -0.18300695436130773))

In [66]:
for i in [.005, 0.001, .05, 0.01, 0.1]:
    print(f"C: {i}")
    print(calculate_confidence_interval(tmp, i))


C: 0.005
(-0.184477655956667, (-0.18589858828000597, -0.18305672363332806))
C: 0.001
(-0.18441025872288805, (-0.18581356308446836, -0.18300695436130773))
C: 0.05
(-0.1844966017113605, (-0.18592156497967308, -0.18307163844304794))
C: 0.01
(-0.18448440217511142, (-0.18590723877338625, -0.1830615655768366))
C: 0.1
(-0.18450656853393899, (-0.1859306687006986, -0.18308246836717937))


In [60]:
for i in [.005, 0.001, .05, 0.01, 0.1]:
    print(f"C: {i}")
    print(calculate_confidence_interval(tmp, i))


C: 0.005
(-0.18460552182849915, (-0.18593459090531855, -0.18327645275167975))
C: 0.001
(-0.1845542190207827, (-0.1858681612585795, -0.1832402767829859))
C: 0.05
(-0.18464953087587355, (-0.1859829802473947, -0.18331608150435239))
C: 0.01
(-0.18462627371845602, (-0.1859573692277028, -0.18329517820920924))
C: 0.1
(-0.18465381117600024, (-0.18598667206435693, -0.18332095028764356))


In [ ]:
#Best parameters found:  {'C': 0.001}
#Best cross-validation accuracy: -0.18469